In [66]:
import pandas as pd
import numpy as np
import openpyxl
from copy import deepcopy

In [18]:
df_liz = pd.read_excel(
    "../data/DataLizen.xlsx",
    sheet_name="Lizenzierte",
    header=2
)
df_liz.head()

,Vereinsname,Geschlecht Person,Altersklasse,Anz. Spieler/innen in Vereinen,Person-Nr anonymisiert
0,FC Adliswil (11001),M,Jun.B,1,8243
1,FC Adliswil (11001),M,Jun.B,1,8318
2,FC Adliswil (11001),W,Jun.C,1,8791
3,FC Adliswil (11001),M,Jun.C,1,10572
4,FC Adliswil (11001),M,Jun.B,1,11055


In [19]:
df_spiel = pd.read_excel(
    "../data/DataLizen.xlsx",
    sheet_name="Spiele",
    header=2
)
df_spiel.head()

,Vereinsname,Teamname,Liga,Begegnung,Position,Anz. Einsätze Spieler/innen,Total Einsatz-Minuten,Anz. Tore,Person-Nr anonymisiert
0,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld linksinnen,1.0,67,NaN,230707
1,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld,NaN,0,NaN,235805
2,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld rechtsinnen,1.0,90,NaN,347144
3,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld linksinnen,1.0,1,NaN,440159
4,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Verteidigung linksaussen,1.0,90,NaN,541906


In [67]:
# df_liz_CD = df_liz[(df_liz["Altersklasse"]=="Jun.C") | (df_liz["Altersklasse"]=="Jun.D")]
df_liz_CD = deepcopy(df_liz)

In [68]:
mask = df_spiel["Person-Nr anonymisiert"].isin(df_liz_CD["Person-Nr anonymisiert"])
df_games = df_spiel[mask].copy()


df_games = pd.merge(df_liz_CD[["Person-Nr anonymisiert", "Geschlecht Person", "Altersklasse"]], df_games, on="Person-Nr anonymisiert", how="right")
df_games.tail()



,Person-Nr anonymisiert,Geschlecht Person,Altersklasse,Vereinsname,Teamname,Liga,Begegnung,Position,Anz. Einsätze Spieler/innen,Total Einsatz-Minuten,Anz. Tore
127666,1233321,W,Jun.D,FC Wülflingen (11264),FC Wülflingen (J-C (F)),Juniorinnen C,30.08.2025 FC Brüttisellen-Dietlikon - FC Wülf...,Mittelfeld,1.0,90,NaN
127667,1321267,W,Jun.C,FC Wülflingen (11264),FC Wülflingen (J-C (F)),Juniorinnen C,30.08.2025 FC Brüttisellen-Dietlikon - FC Wülf...,Innenverteidigung rechts,1.0,45,NaN
127668,1327725,W,Jun.D,FC Wülflingen (11264),FC Wülflingen (J-C (F)),Juniorinnen C,30.08.2025 FC Brüttisellen-Dietlikon - FC Wülf...,Mittelsturm,1.0,90,NaN
127669,1358673,W,Jun.D,FC Wülflingen (11264),FC Wülflingen (J-C (F)),Juniorinnen C,30.08.2025 FC Brüttisellen-Dietlikon - FC Wülf...,Flügel rechts,1.0,45,NaN
127670,1366503,W,Jun.D,FC Wülflingen (11264),FC Wülflingen (J-C (F)),Juniorinnen C,30.08.2025 FC Brüttisellen-Dietlikon - FC Wülf...,Ersatz (S),1.0,56,NaN


In [70]:
list_Altersklasse = (df_games["Altersklasse"]
                 .str.split(",")
                 .explode()
                 .str.strip()
                 .dropna()
                 .unique())
print(list_Altersklasse)

['Jun.A' 'Aktiv' 'Jun.B' '30+' 'Jun.C' 'Jun.D' '50+' '40+']


In [80]:
# the NaN goals, it is 0 goals, must be all integers
df_games["Anz. Tore"] = (
    pd.to_numeric(df_games["Anz. Tore"], errors="coerce")
      .fillna(0)
      .astype(int)
)

# list of best players
players_list = (
    df_games.groupby("Person-Nr anonymisiert", as_index=False).agg({
        "Anz. Tore": "sum",
        "Total Einsatz-Minuten": "sum",
        "Altersklasse":"last", 
        "Liga": lambda x: ", ".join(sorted(set(x.dropna()))),
        "Position": lambda x: ", ".join(sorted(set(x.dropna()))), 
        "Vereinsname": "last", 
        "Geschlecht Person": "first"
        })
        .rename(columns={"Anz. Tore": "Goals", "Person-Nr anonymisiert" : "Id-Player", "Total Einsatz-Minuten":"Minutes", "Geschlecht Person":"Sex"})
        .sort_values("Goals", ascending=False)
)
print(players_list.shape)
players_list.head()

(20566, 8)


,Id-Player,Goals,Minutes,Altersklasse,Liga,Position,Vereinsname,Sex
13672,954309,51,810,Jun.C,"Juniorinnen B, Juniorinnen C","Flügel links, Sturmspitze links, Sturmspitze r...",FC Embrach (11266),W
7626,539951,30,786,Jun.C,"Junioren B 1. Stärkeklasse, Junioren C 1. Stär...","Flügel links, Mittelfeld, Mittelfeld linksauss...",FC Dielsdorf (11008),M
5888,416448,27,1186,Jun.C,"Junioren A Promotion, Junioren C 1. Stärkeklasse","Mittelsturm, Sturmspitze, Tor",FC Mönchaltorf (11035),M
9451,666237,26,554,Jun.C,Junioren C 2. Stärkeklasse,"Mittelsturm, Sturmspitze, Sturmspitze links, S...",SV Seebach ZH (11488),M
18179,1269933,26,785,Jun.D,Junioren C 2. Stärkeklasse,"Flügel links, Sturmspitze links",FC Mönchaltorf (11035),M


In [97]:
club_top = (
    players_list
    .groupby("Vereinsname", as_index=False)
    .agg(
        Cant_Player=("Id-Player", "count"),
        Cant_Goal=("Goals", "sum"),
        Altersklasse=("Altersklasse", lambda x: ", ".join(sorted(set(x.dropna())))),
    )
    .sort_values("Cant_Goal", ascending=False)
)
club_top

,Vereinsname,Cant_Player,Cant_Goal,Altersklasse
119,FC Witikon (11496),386,498,"30+, 40+, 50+, Aktiv, Jun.A, Jun.B, Jun.C, Jun.D"
35,FC Embrach (11266),304,444,"30+, 40+, 50+, Aktiv, Jun.A, Jun.B, Jun.C, Jun.D"
116,FC Wiedikon ZH (11494),359,419,"30+, 40+, 50+, Aktiv, Jun.A, Jun.B, Jun.C, Jun.D"
115,FC Wetzikon (11055),279,417,"30+, 40+, 50+, Aktiv, Jun.A, Jun.B, Jun.C, Jun.D"
51,FC Herrliberg (11057),320,416,"30+, 40+, 50+, Aktiv, Jun.A, Jun.B, Jun.C, Jun.D"
...,...,...,...,...
126,FFC Lionesses (11456),20,12,"40+, Jun.A, Jun.B, Jun.C"
11,FC Augwil (11255),23,9,"30+, Aktiv, Jun.A, Jun.B"
2,Benfica Clube de Zurique (11536),27,9,"30+, 40+, Aktiv, Jun.A, Jun.B"
104,FC Turkuaz Zürich (11535),25,8,"30+, 40+, Aktiv, Jun.A, Jun.B"


In [ ]:
print(players_list.columns.to_list())
print(club_top.columns.to_list())
players_list.head()

['Id-Player', 'Goals', 'Minutes', 'Altersklasse', 'Liga', 'Position', 'Vereinsname', 'Sex']
['Vereinsname', 'Cant_Player', 'Cant_Goal', 'Altersklasse']


In [99]:
club_top.to_csv("../data/club_top.csv", index=False, encoding="utf-8") 

players_list.to_csv("../data/players_list.csv", index=False, encoding="utf-8") 


In [44]:
club_top2 = pd.read_csv("../data/club_top.csv", encoding="utf-8")
club_top = club_top2.copy()
club_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Vereinsname  113 non-null    object
 1   Cant_Player  113 non-null    int64 
 2   Cant_Goal    113 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.8+ KB


In [93]:
players_list2 = pd.read_csv("../data/players_list.csv", encoding="utf-8")
players_list =players_list2.copy()

In [92]:
players_list.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20566 entries, 13672 to 20544
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id-Player     20566 non-null  int64 
 1   Goals         20566 non-null  int64 
 2   Minutes       20566 non-null  int64 
 3   Altersklasse  20566 non-null  object
 4   Liga          20566 non-null  object
 5   Position      20566 non-null  object
 6   Vereinsname   20566 non-null  object
 7   Sex           20566 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.4+ MB


In [91]:
Verein ="FC Dielsdorf (11008)"
df_filt = players_list[players_list["Vereinsname"]==Verein]
mask =df_filt["Goals"].max()
best_player = df_filt[df_filt["Goals"] == mask]
print(best_player)

print(best_player["Id-Player"])

      Id-Player  Goals  Minutes Altersklasse  \
7626     539951     30      786        Jun.C   

                                                   Liga  \
7626  Junioren B 1. Stärkeklasse, Junioren C 1. Stär...   

                                               Position           Vereinsname  \
7626  Flügel links, Mittelfeld, Mittelfeld linksauss...  FC Dielsdorf (11008)   

     Sex  
7626   M  
7626    539951
Name: Id-Player, dtype: int64


In [90]:
list_position = (players_list2["Position"]
                 .str.split(",")
                 .explode()
                 .str.strip()
                 .dropna()
                 .unique())
print(list_position)

['Flügel links' 'Sturmspitze links' 'Sturmspitze rechts' 'Mittelfeld'
 'Mittelfeld linksaussen' 'Mittelsturm' 'Sturmspitze' 'Tor' 'Ersatz (S)'
 'Flügel rechts' 'Mittelfeld defensiv' 'Mittelfeld defensiv links'
 'Mittelfeld rechtsinnen' 'hängende Spitze' 'Mittelfeld rechts'
 'Innenverteidigung links' 'Libero' 'Verteidigung mitte'
 'hängende Spitze links' 'Hängende Spitze rechts' 'Mittelfeld linksinnen'
 'Mittelfeld links' 'Verteidigung rechts' 'Innenverteidigung rechts'
 'Mittelfeld rechtsaussen' 'Verteidigung linksaussen' 'Verteidigung links'
 'Verteidigung vorne' 'Mittelfeld defensiv rechts'
 'Verteidigung rechtsaussen' '?' 'Ersatz (T)']


In [83]:
list_Liga = (players_list2["Liga"]
                 .str.split(",")
                 .explode()
                 .str.strip()
                 .dropna()
                 .unique())
print(list_Liga)

['Juniorinnen B' 'Juniorinnen C' 'Junioren B 1. Stärkeklasse'
 'Junioren C 1. Stärkeklasse' 'Junioren A Promotion'
 'Junioren C 2. Stärkeklasse' 'Junioren B 2. Stärkeklasse'
 'Junioren B Promotion' 'Junioren C Promotion' 'Youth League C'
 'Frauen 4. Liga' 'U-16 (Frauen)' 'Frauen 3. Liga' '4. Liga'
 'Youth League B' 'U-15' 'Junioren A' '5. Liga' '3. Liga' 'Frauen 2. Liga'
 'U-18 (Frauen)' "U-18 Women's Cup" 'Nationalliga B (Frauen)']


In [89]:
mask2 = df_spiel["Person-Nr anonymisiert"].isin(df_liz["Person-Nr anonymisiert"]).copy()
df_games_all = df_spiel[mask2].copy()

df_games_all = pd.merge(df_liz[["Person-Nr anonymisiert", "Geschlecht Person","Altersklasse"]], df_games_all, on="Person-Nr anonymisiert", how="right")


df_games_all["Anz. Tore"] = (
    pd.to_numeric(df_games_all["Anz. Tore"], errors="coerce")
      .fillna(0)
      .astype(int)
)


# list of best players
all_players = (
    df_games_all.groupby("Person-Nr anonymisiert", as_index=False).agg({
        "Anz. Tore": "sum",
        "Total Einsatz-Minuten": "sum",
        "Altersklasse":"last", 
        "Liga": lambda x: ", ".join(sorted(set(x.dropna()))),
        "Position": lambda x: ", ".join(sorted(set(x.dropna()))), 
        "Vereinsname": "last", 
        "Geschlecht Person": "first"
        })
        .rename(columns={
            "Anz. Tore": "Goals", "Person-Nr anonymisiert" : "Id-Player", 
            "Total Einsatz-Minuten":"Minutes", "Geschlecht Person":"Sex"})
        .sort_values("Goals", ascending=False)
)
all_players.info()

all_players.drop(columns=["Goals", "Minutes", "Position","Sex"], inplace=True)
all_players.head()
all_players.info()
print(all_players.columns.to_list())
all_players.to_csv("../data/all_players.csv", index=False, encoding="utf-8") 



<class 'pandas.core.frame.DataFrame'>
Index: 20566 entries, 13672 to 20544
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id-Player     20566 non-null  int64 
 1   Goals         20566 non-null  int64 
 2   Minutes       20566 non-null  int64 
 3   Altersklasse  20566 non-null  object
 4   Liga          20566 non-null  object
 5   Position      20566 non-null  object
 6   Vereinsname   20566 non-null  object
 7   Sex           20566 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.4+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 20566 entries, 13672 to 20544
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id-Player     20566 non-null  int64 
 1   Altersklasse  20566 non-null  object
 2   Liga          20566 non-null  object
 3   Vereinsname   20566 non-null  object
dtypes: int64(1), object(3)
memory usage: 803.4+ KB
['Id-Player'

In [50]:
list_Liga = (players_list["Liga"]
                 .str.split(",")
                 .explode()
                 .str.strip()
                 .dropna()
                 .unique())
list_Liga

array(['Juniorinnen B', 'Juniorinnen C', 'Junioren B 1. Stärkeklasse',
       'Junioren C 1. Stärkeklasse', 'Junioren A Promotion',
       'Junioren C 2. Stärkeklasse', 'Junioren B 2. Stärkeklasse',
       'Junioren B Promotion', 'Junioren C Promotion', 'Youth League C',
       'Frauen 4. Liga', 'U-16 (Frauen)', 'Frauen 3. Liga', '4. Liga',
       'Youth League B', 'U-15', 'Junioren A', '5. Liga', '3. Liga',
       'Frauen 2. Liga', 'U-18 (Frauen)', "U-18 Women's Cup",
       'Nationalliga B (Frauen)'], dtype=object)

In [88]:
list_Alterklasse = (players_list["Altersklasse"]
                 .str.strip()
                 .dropna()
                 .unique())
list_Alterklasse

array(['Jun.C', 'Jun.D', 'Aktiv', '30+', 'Jun.B', 'Jun.A', '50+', '40+'],
      dtype=object)